# Reading, Scraping, and Cleaning Our Data

This notebook shows our process of data munging in more detail.

## Packages

In [2]:
# Loading in data:
import numpy as np
import pandas as pd

# Parsing:
import requests
import requests_cache
import lxml
from bs4 import BeautifulSoup
import bs4
import re

# I. Reading in the Data - Our Primary Data Source

The following cell loads are data through read_csv and takes a long time to run, about 15 minutes on our devices. We use street.h5 to import the data faster after importing the csv once here.

In [ ]:
# SHould only run the first time ever using this notebook:
parseDates = ["Opened", "Closed", "Updated"] # Convert these to datetimes
street_csv = pd.read_csv("Street_and_Sidewalk_Cleaning.csv", 
                         #nrows = 100000,
                         parse_dates=parseDates)
street.to_hdf('street.h5','table',append=False) # Convert of h5 for faster readining in later

The next cell loads the h5 and displays the first few rows of our primary data source. All of the variables can be seen here.

In [3]:
# Write h5: Run every second time and on of using this notebook
street_hdf = pd.HDFStore('street.h5')
street = street_hdf.select('/table')
street.head()

,CaseID,Opened,Closed,Status,Responsible Agency,Address,Category,Request Type,Request Details,Source,Supervisor District,Neighborhood,Updated,Point
0,322566,2008-11-30 22:56:00,2009-07-21 16:24:00,Closed,DPW Ops Queue,"1566 HYDE ST, SAN FRANCISCO, CA, 94109",Street and Sidewalk Cleaning,Sidewalk_Cleaning,Garbage,Voice In,3.0,Nob Hill,2009-07-21 16:24:00,"(37.795328529, -122.418067787)"
1,322560,2008-11-30 22:24:00,2009-07-21 16:24:00,Closed,DPW Ops Queue,Intersection of TAYLOR ST and TURK ST,Street and Sidewalk Cleaning,Illegal_Dumping,Garbage,Voice In,6.0,Downtown/Civic Center,2009-07-21 16:24:00,"(37.7832142923006, -122.410764953722)"
2,322559,2008-11-30 22:20:00,2008-12-01 06:07:00,Closed,DPW Ops Queue,"1601 SACRAMENTO ST, SAN FRANCISCO, CA, 94109",Street and Sidewalk Cleaning,Illegal_Dumping,Mattress,Voice In,3.0,Nob Hill,2008-12-01 06:07:00,"(37.791643509125, -122.419260423291)"
3,322556,2008-11-30 21:34:00,2008-12-09 08:07:00,Closed,DPW Ops Queue,Intersection of 25TH ST and LUCKY ST,Street and Sidewalk Cleaning,Sidewalk_Cleaning,Human_waste_or_urine,Voice In,9.0,Mission,2008-12-09 08:07:00,"(37.7509362479361, -122.413364630104)"
4,322545,2008-11-30 20:45:00,2009-02-03 07:09:00,Closed,DPW Ops Queue,Intersection of 19TH ST and OAKWOOD ST,Street and Sidewalk Cleaning,Sidewalk_Cleaning,Empty_cart,Voice In,8.0,Mission,2009-02-03 07:09:00,"(37.7598935230725, -122.42486308879)"


## Cleaning the data

To keep counts of requests consistent over the months and days, we remove years with incomplete data since our original data set starts in November 2008 and ends in January 2017.  

In [4]:
street = street.loc[street['Opened'].dt.year != 2008]
street = street.loc[street['Opened'].dt.year != 2017]
street = street.sort_values("Opened") # Re-sort data in date order
street = street.reset_index() # Re-set the indexing
street = street.drop('index', 1) # Get rid of old index column 

Some neighborhood names need to be changed to allow merging with other data sources later and to match our shape files.   

In [5]:
street.ix[street.Neighborhood == "None", "Neighborhood"] = np.nan
street.ix[street.Neighborhood == "Ocean View", "Neighborhood"] = "Oceanview"
street.ix[street.Neighborhood == "Downtown/Civic Center", "Neighborhood"] = "Civic Center"
street.ix[street.Neighborhood == "St. Mary's Park", "Neighborhood"] = "St. Marys Park"
street.ix[street.Neighborhood == "Presidio", "Neighborhood"] = "Presidio Heights"

Next we create a month column for future plots and data merging. 

In [6]:
street['month'] = [timestamp.month for timestamp in street.Opened]

A look of the first few rows of our cleaned data:

In [8]:
street.head()

,CaseID,Opened,Closed,Status,Responsible Agency,Address,Category,Request Type,Request Details,Source,Supervisor District,Neighborhood,Updated,Point,month
0,342509,2009-01-01 08:30:51,2009-01-01 11:07:06,Closed,DPW Ops Queue,Intersection of 13TH ST and FOLSOM ST,Street and Sidewalk Cleaning,Sidewalk_Cleaning,Encampment,Voice In,6.0,Mission,2009-01-01 11:07:06,"(37.7695911772607, -122.415577110949)",1
1,342510,2009-01-01 08:33:46,2009-01-01 11:07:06,Closed,DPW Ops Queue,Intersection of 13TH ST and FOLSOM ST,Street and Sidewalk Cleaning,Sidewalk_Cleaning,Debris_filled_carts,Voice In,6.0,Mission,2009-01-01 11:07:06,"(37.7695911772607, -122.415577110949)",1
2,342512,2009-01-01 08:44:54,2009-01-31 13:09:53,Closed,DPW Ops Queue,"467 FILLMORE ST, SAN FRANCISCO, CA, 94117",Street and Sidewalk Cleaning,Street_Cleaning,Glass,Voice In,5.0,Western Addition,2009-01-31 13:09:53,"(37.773807246, -122.431027495)",1
3,342514,2009-01-01 09:13:07,2009-01-01 11:07:06,Closed,DPW Ops Queue,Intersection of DWIGHT ST and GOETTINGEN ST,Street and Sidewalk Cleaning,Sidewalk_Cleaning,Garbage,Voice In,9.0,Excelsior,2009-01-01 11:07:06,"(37.7232896018615, -122.405086927628)",1
4,342519,2009-01-01 09:21:05,2009-01-21 06:07:13,Closed,DPW Ops Queue,"1610 MCALLISTER ST, SAN FRANCISCO, CA, 94115",Street and Sidewalk Cleaning,Sidewalk_Cleaning,Human_waste_or_urine,Voice In,5.0,Western Addition,2009-01-21 06:07:13,"(37.777956377, -122.43893262)",1


# II. Scraping - Demographic Data

In [10]:
requests_cache.install_cache('sf_cache')

In [11]:
url = "http://www.city-data.com/nbmaps/neigh-San-Francisco-California.html"
response = requests.get(url)
response.raise_for_status

neighborhoods_bs = BeautifulSoup(response.text, 'lxml')
neighborhood_names = neighborhoods_bs.find_all(name = "span", attrs={'class':'street-name'})
neighborhood_names = [name.text.replace("-", " ") for name in neighborhood_names]
neighborhood_divs = neighborhoods_bs.body.find_all(name = "div", attrs={'class':'neighborhood'})

In [12]:
def catch(line, number, simple, c = "h", complicated = False):
    """
    This function was made to catch the cases when scraping where we want NAs. If grabbing what we want from line fails, 
    then NA is returned. If number is True, then the value we are grabbing needs to be converted from a string to a number. 
    If simple is True, then the value we are scraping is right there, otherwise we need to search for a particular class which
    is the c input variable. Complicated is for the last column we are scraping where finding that class is a bit more complicated.
    """
    try:
        if number == True:
            if simple == True:
                return float(line[0].next_sibling.replace(",", "").replace("$", ""))
            
            else: 
                if complicated == False:
                    return float(line[0].next_sibling.next_sibling.find_all(class_ = c)[0].next_sibling.replace(",", "").replace(" years", "").replace("$", ""))
                else: 
                    return float(line[0].find_all_next(class_ = c)[0].next_sibling.replace(",", "").replace(" years", "").replace("$", ""))

        else:
            return line[0].next_sibling
    except:
        return np.nan

In [13]:
demographic = pd.DataFrame({
        'Neighborhood': neighborhood_names, 
        'AreaSqMi': [catch(neigh.find_all(name = "b", string = "Area:"), True, True) for neigh in neighborhood_divs] ,
        'Population': [catch(neigh.find_all(name = "b", string = "Population:"), True, True) for neigh in neighborhood_divs], 
        'PeoplePerSqMi': [catch(neigh.find_all(name = "b", string = "Population density:"), True, False) for neigh in neighborhood_divs],
        'MedHouseholdIncome': [catch(neigh.find_all(name = "b", string = "Median household income in 2015: "), True, False) for neigh in neighborhood_divs],
        'MedRent': [catch(neigh.find_all(name = "b", string = "Median rent in in 2015: "), True, False) for neigh in neighborhood_divs],
        'Males': [catch(neigh.find_all(name = "b", string = "Male vs Females"), True, False) for neigh in neighborhood_divs],
        'Females': [catch(neigh.find_all(name = "b", string = "Male vs Females"), True, False, "a") for neigh in neighborhood_divs], 
        'MedAgeM': [catch(neigh.find_all(name = "b", string = "Median age"), True, False) for neigh in neighborhood_divs],
        'MedAgeF': [catch(neigh.find_all(name = "b", string = "Median age"), True, False, "a") for neigh in neighborhood_divs],
        'HousePrice': [catch(neigh.find_all(name = "b", string = "Average estimated value of detached houses in 2015 "), True, False, complicated = True) for neigh in neighborhood_divs],
})

demographic.head()

,AreaSqMi,Females,HousePrice,Males,MedAgeF,MedAgeM,MedHouseholdIncome,MedRent,Neighborhood,PeoplePerSqMi,Population
0,0.144,2461.0,1988926.0,3916.0,38.6,35.4,93901.0,1754.0,Alamo Square,44418.0,6379.0
1,0.124,944.0,455053.0,1177.0,38.9,41.2,104697.0,1937.0,Anza Vista,17080.0,2122.0
2,0.037,96.0,1101672.0,145.0,48.2,48.4,139017.0,2014.0,Aquatic Park,6487.0,241.0
3,0.055,774.0,NaN,1365.0,35.2,35.1,134523.0,2489.0,Baja Noe,38816.0,2141.0
4,0.048,312.0,NaN,377.0,42.6,38.6,84685.0,1591.0,Balboa Park,14350.0,690.0


In [14]:
def CombineNeighs(name1, name2, newName, df):
    """
    This function takes in two neighborhood names of the demographic dataframe, combines them, 
    and changes the name.
    """
    one = df.ix[df.Neighborhood == name1]
    two = df.ix[df.Neighborhood == name2]
    one.loc[one.index, "AreaSqMi"] = (one['AreaSqMi'].values[0] + two['AreaSqMi'].values[0]) / 2
    one.loc[one.index, 'Females'] = one['Females'].values[0] + two['Females'].values[0]
    one.loc[one.index, 'HousePrice'] = (one['HousePrice'].values[0] + two['HousePrice'].values[0]) / 2
    one.loc[one.index, 'Males'] = one['Males'].values[0] + two['Males'].values[0]
    one.loc[one.index, 'MedAgeF'] = (one['MedAgeF'].values[0] + two['MedAgeF'].values[0]) / 2
    one.loc[one.index, 'MedAgeM'] = (one['MedAgeM'].values[0] + two['MedAgeM'].values[0]) / 2
    one.loc[one.index, 'MedHouseholdIncome'] = (one['MedHouseholdIncome'].values[0] + two['MedHouseholdIncome'].values[0]) / 2
    one.loc[one.index, 'PeoplePerSqMi'] = (one['PeoplePerSqMi'].values[0] + two['PeoplePerSqMi'].values[0]) / 2
    one.loc[one.index, 'Population'] = one['Population'].values[0] + two['Population'].values[0]   
    one.loc[one.index, 'Neighborhood'] = newName
    df = df.drop(one.index)
    df = df.drop(two.index)
    df = df.append(one)
    return df

In [15]:
# Clean up neighborhood names: make the demographic names match our data and shapefiles
demographic = CombineNeighs("Bayview District", "Bayview Heights", "Bayview", demographic)
demographic = CombineNeighs("Bernal Heights North", "Bernal Heights Park", "Bernal Heights1", demographic)
demographic = CombineNeighs("Bernal Heights South", "Bernal Heights", "Bernal Heights2", demographic)
demographic = CombineNeighs("Bernal Heights1", "Bernal Heights2", "Bernal Heights", demographic)
demographic = CombineNeighs("Balboa Park", "Balboa Terrace", "Balboa Terrace", demographic)
demographic = CombineNeighs("Aquatic Park", "Fort Mason", 'Aquatic Park / Ft. Mason', demographic)
demographic = CombineNeighs("Downtown", "Union Square", 'Downtown / Union Square', demographic)
demographic = CombineNeighs("Financial District", "Financial District South", "Financial District", demographic)


/home/tgordon/.local/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [16]:
demographic.loc[demographic.Neighborhood == "Buena Vista Park", "Neighborhood"] = "Buena Vista"
demographic.loc[demographic.Neighborhood == "Cayuga Terrace", "Neighborhood"] = "Cayuga"
demographic.loc[demographic.Neighborhood == "Ingleside Terrace", "Neighborhood"] = "Ingleside Terraces"
demographic.loc[demographic.Neighborhood == "Laurel Heights", "Neighborhood"] = 'Laurel Heights / Jordan Park'
demographic.loc[demographic.Neighborhood == "Lake Shore", "Neighborhood"] = 'Lakeshore'
demographic.loc[demographic.Neighborhood == "South Of Market", "Neighborhood"] = 'South of Market'
demographic.loc[demographic.Neighborhood == "North Waterfront", "Neighborhood"] = 'Northern Waterfront'
demographic.loc[demographic.Neighborhood == "St Marys Square", "Neighborhood"] = "St. Marys Park"
demographic.loc[demographic.Neighborhood == "Saint Francis Wood", "Neighborhood"] = "St. Francis Wood"
demographic.loc[demographic.Neighborhood == "Sea Cliff", "Neighborhood"] = "Seacliff"
demographic.loc[demographic.Neighborhood == "Parnassus", "Neighborhood"] = "Parnassus Heights"
demographic.loc[demographic.Neighborhood == "Park Merced", "Neighborhood"] = "Parkmerced"
demographic.loc[demographic.Neighborhood == "Mission District", "Neighborhood"] = "Mission"
demographic.loc[demographic.Neighborhood == "Marina District", "Neighborhood"] = "Marina"
demographic.loc[demographic.Neighborhood == "Mount Davidson Manor", "Neighborhood"] = "Mt. Davidson Manor"
demographic.loc[demographic.Neighborhood == "Lake", "Neighborhood"] = "Lake Street"

In [17]:
# set(street.Neighborhood.unique()) - set(demographic.Neighborhood.unique())

In [18]:
demographic

,AreaSqMi,Females,HousePrice,Males,MedAgeF,MedAgeM,MedHouseholdIncome,MedRent,Neighborhood,PeoplePerSqMi,Population
0,0.1440,2461.0,1988926.0,3916.0,38.600,35.40,93901.0,1754.0,Alamo Square,44418.00,6379.0
1,0.1240,944.0,455053.0,1177.0,38.900,41.20,104697.0,1937.0,Anza Vista,17080.00,2122.0
3,0.0550,774.0,NaN,1365.0,35.200,35.10,134523.0,2489.0,Baja Noe,38816.00,2141.0
12,0.1380,1009.0,1423695.0,1492.0,39.400,41.70,144714.0,1813.0,Buena Vista,18167.00,2502.0
13,0.5260,5261.0,1843918.0,7671.0,38.400,41.70,142771.0,1946.0,Castro,24605.00,12935.0
14,0.0950,1446.0,NaN,1510.0,58.100,47.30,64491.0,1383.0,Cathedral Hill,31073.00,2956.0
15,0.1920,2745.0,1040534.0,2383.0,45.300,39.60,75063.0,1517.0,Cayuga,26690.00,5128.0
16,0.8580,13534.0,778920.0,12590.0,46.400,40.00,79040.0,1732.0,Central Richmond,30442.00,26124.0
17,0.9340,11010.0,855173.0,9564.0,42.700,40.60,120276.0,1864.0,Central Sunset,22023.00,20573.0
18,0.1680,4496.0,NaN,4703.0,49.400,43.70,33769.0,689.0,Chinatown,54745.00,9200.0


In [45]:
# Export demographic here, then import in the other notebook and merge before plotting.  
demographic.to_csv("demographic.csv")

--------

# III. Scraping - Events and Festivals

In [20]:
url_pride = "https://en.wikipedia.org/wiki/San_Francisco_Pride"
response = requests.get(url_pride)
response.raise_for_status

pride_bs = BeautifulSoup(response.text, 'lxml')

pride_table = pride_bs.find_all(name = "table", attrs={'class':'wikitable'})


In [21]:
#type(pride_table[0])
# The table is the first element

rows = pride_table[0].find_all(name = "tr")

# The first row is the header
#print rows[0]
#for colname in rows[0].find_all(name = "th"):
#    print colname.text

colnames = [colname.text for colname in rows[0].find_all(name = "th")]
print colnames

#pride = [{colname, []} for colname in colnames]
#print pride

[u'Year', u'Dates', u'Festival name', u'Theme', u'Estimated attendance', u'Notes']


In [22]:
years = []
dates = []
attendance = []

for row in rows[1:]:
    cells = row.find_all(name = "td")
        
    years.append(cells[0].text)
    dates.append(cells[1].text)
    attendance.append(cells[4].text)
    
assert(len(years) == len(dates) == len(attendance))

In [23]:
pride = pd.DataFrame.from_dict({"year": years,
                                "date": dates,
                                "attendance": attendance})

pride.year = pd.to_numeric(pride.year)

# Remove unused years
pride = pride[pride.year > 2007]

pride

,attendance,date,year
38,1.2 million[25],June 28–29,2008
39,1.2 million[26],June 27–28,2009
40,1.2 million[27],June 26–27,2010
41,1 million[28],June 25–26,2011
42,,June 23–24,2012
43,1.5 Million,June 29–30,2013
44,1.7 million,June 28–29,2014
45,1.8 million,June 27–28,2015
46,TBD,June 25-26,2016


In [25]:
startdates = [d[:-3] for d in pride.date]
enddates = [d[:4] + d[-2:] for d in pride.date]
startdatetimes = []
enddatetimes = []

for sdate, edate, year in zip(startdates, enddates, pride.year):
    startdatetimes.append(sdate + " " + str(year))
    enddatetimes.append(edate + " " + str(year))
    
pride["start_date"] = pd.to_datetime(startdatetimes)
pride["end_date"] = pd.to_datetime(enddatetimes)
#pride["datetimes"] = [pd.date_range(dt, periods=2) for dt in startdatetimes]
pride

,attendance,date,year,start_date,end_date
38,1.2 million[25],June 28–29,2008,2008-06-28,2008-06-29
39,1.2 million[26],June 27–28,2009,2009-06-27,2009-06-28
40,1.2 million[27],June 26–27,2010,2010-06-26,2010-06-27
41,1 million[28],June 25–26,2011,2011-06-25,2011-06-26
42,,June 23–24,2012,2012-06-23,2012-06-24
43,1.5 Million,June 29–30,2013,2013-06-29,2013-06-30
44,1.7 million,June 28–29,2014,2014-06-28,2014-06-29
45,1.8 million,June 27–28,2015,2015-06-27,2015-06-28
46,TBD,June 25-26,2016,2016-06-25,2016-06-26


In [36]:
def parse_attendance(attendance_str):
    """
    Parse the attendance strings in the table in the Wikipedia page for the Pride parade. 
    Returns a float or None if the string cannot be parsed.
    """
    
    if "million" in attendance_str.lower():
        # Multiply by 1000000 and remove the string "million"
        return float(attendance_str.split(" ")[0]) * 1000000
    
    else:
        return None

        
pride["attendance_num"] = [parse_attendance(x) for x in pride.attendance]
pride = pride.reset_index()
pride

,level_0,index,attendance,date,year,start_date,end_date,attendance_num,StartNoTime,EndNoTime
0,0,38,1.2 million[25],June 28–29,2008,2008-06-28,2008-06-29,1200000.0,2008-06-28,2008-06-29
1,1,39,1.2 million[26],June 27–28,2009,2009-06-27,2009-06-28,1200000.0,2009-06-27,2009-06-28
2,2,40,1.2 million[27],June 26–27,2010,2010-06-26,2010-06-27,1200000.0,2010-06-26,2010-06-27
3,3,41,1 million[28],June 25–26,2011,2011-06-25,2011-06-26,1000000.0,2011-06-25,2011-06-26
4,4,42,,June 23–24,2012,2012-06-23,2012-06-24,NaN,2012-06-23,2012-06-24
5,5,43,1.5 Million,June 29–30,2013,2013-06-29,2013-06-30,1500000.0,2013-06-29,2013-06-30
6,6,44,1.7 million,June 28–29,2014,2014-06-28,2014-06-29,1700000.0,2014-06-28,2014-06-29
7,7,45,1.8 million,June 27–28,2015,2015-06-27,2015-06-28,1800000.0,2015-06-27,2015-06-28
8,8,46,TBD,June 25-26,2016,2016-06-25,2016-06-26,NaN,2016-06-25,2016-06-26


### Outside Lands Music and Arts Festival

In [4]:
url_ol = "https://en.wikipedia.org/wiki/Outside_Lands_Music_and_Arts_Festival"
response = requests.get(url_ol)
response.raise_for_status

ol_bs = BeautifulSoup(response.text, 'lxml')


# Years are in h3
h3 = ol_bs.find_all(name = "h3")

ol = []

for h in h3:
    span = h.find_all(name = "span", attrs={"class":"mw-headline"})
    
    # If there is a span in the h3
    if span:
        year = span[0].text
        #print year 
        
        check_p = True
        
        #dates = []
        
        for sibling in h.find_next_siblings(limit=5):
            
            # Days are in h4 or p
            if sibling.name == "h4":
                #print "h4" + "\t" + sibling.text
                ol.append([year, sibling.text.replace(u"\u2013", "-").replace("[edit]","")])
                # If an h4 was found, stop looking for p
                check_p = False
                
            elif sibling.name == "p" and check_p:
                #print check_p
                #print "p" + "\t" + sibling.text
                ol.append([year, sibling.text.replace(u"\u2013", "-")])
                # Formatting for 2011+ uses the date in a p tag
                if "August" in sibling.text:
                    break
                
        #print "\n"
        #ol[str(year)] = dates

ol

[[u'2008', u'Friday August 22'],
 [u'2008', u'Saturday August 23'],
 [u'2008', u'Sunday August 24'],
 [u'2009', u'Friday August 28'],
 [u'2009', u'Saturday August 29'],
 [u'2010', u'Saturday August 14'],
 [u'2010', u'Sunday August 15'],
 [u'2011', u'August 12-14'],
 [u'2012', u'August 10-12'],
 [u'2013', u'August 9-11'],
 [u'2014', u'August 8-10'],
 [u'2015', u'August 7-9'],
 [u'2016', u'August 5-9']]

In [5]:
# Separate the date ranges and fix the formatting

ol2 = []
for year_and_date in ol:
    
    date_split = year_and_date[1].split()
    month = date_split[0]
    if len(date_split) > 2:
        # keep the year
        year_and_date_new = [year_and_date[0]]
        year_and_date_new.extend(date_split[1:])
        ol2.append(year_and_date_new)
    else:
        days = date_split[1].split("-")
        for day in days:
            # keep the year
            year_and_date_new = [year_and_date[0]]
            year_and_date_new.append(month)
            year_and_date_new.append(day)
            ol2.append(year_and_date_new)

ol2

[[u'2008', u'August', u'22'],
 [u'2008', u'August', u'23'],
 [u'2008', u'August', u'24'],
 [u'2009', u'August', u'28'],
 [u'2009', u'August', u'29'],
 [u'2010', u'August', u'14'],
 [u'2010', u'August', u'15'],
 [u'2011', u'August', u'12'],
 [u'2011', u'August', u'14'],
 [u'2012', u'August', u'10'],
 [u'2012', u'August', u'12'],
 [u'2013', u'August', u'9'],
 [u'2013', u'August', u'11'],
 [u'2014', u'August', u'8'],
 [u'2014', u'August', u'10'],
 [u'2015', u'August', u'7'],
 [u'2015', u'August', u'9'],
 [u'2016', u'August', u'5'],
 [u'2016', u'August', u'9']]

In [6]:
ol_dates = pd.to_datetime([" ".join(date) for date in ol2])
ol_dates

DatetimeIndex(['2008-08-22', '2008-08-23', '2008-08-24', '2009-08-28',
               '2009-08-29', '2010-08-14', '2010-08-15', '2011-08-12',
               '2011-08-14', '2012-08-10', '2012-08-12', '2013-08-09',
               '2013-08-11', '2014-08-08', '2014-08-10', '2015-08-07',
               '2015-08-09', '2016-08-05', '2016-08-09'],
              dtype='datetime64[ns]', freq=None)

In [10]:
ol_dates_df = pd.DataFrame({"Festival_Date": ol_dates})
ol_dates_df.to_csv("ol_dates.csv")

# Events Plots

Grab all of June and group by day to get count of requests, then add boolean pride or not

In [31]:
streetJune = street.loc[street["Opened"].dt.month == 6]
streetJune["DateOpened"] = streetJune["Opened"].dt.date # Yells at you

# http://www.sfpride.org/parade/
prideNeighs = ["South of Market", "Tenderloin", "Financial District", "Downtown / Union Square", "Civic Center"]
streetJune = streetJune.loc[streetJune.Neighborhood.isin(prideNeighs)]

JuneDayReqs = streetJune.groupby(by = "DateOpened").count().CaseID
pride["StartNoTime"] = pride["start_date"].dt.date 
pride["EndNoTime"] = pride["end_date"].dt.date 
JuneRequests = pd.DataFrame({"ReqCount": JuneDayReqs})
JuneRequests = JuneRequests.reset_index()
JuneRequests["Pride"] = [row in pride.StartNoTime.values for row in JuneRequests.DateOpened]

/home/tgordon/.local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [32]:
streetJune.head()

,CaseID,Opened,Closed,Status,Responsible Agency,Address,Category,Request Type,Request Details,Source,...,Females,HousePrice,Males,MedAgeF,MedAgeM,MedHouseholdIncome,MedRent,PeoplePerSqMi,Population,DateOpened
19953,441365,2009-06-01 05:59:27,2009-06-03 06:11:11,Closed,DPW Ops Queue,"34 SHERIDAN ST, SAN FRANCISCO, CA, 94103",Street and Sidewalk Cleaning,Illegal_Dumping,Furniture,Voice In,...,4643.0,NaN,6977.0,40.00,39.60,92455.0,1047.0,18297.0,11623.0,2009-06-01
19954,441366,2009-06-01 06:04:06,2009-06-02 08:10:47,Closed,DPW Ops Queue,Intersection of 6TH ST and NATOMA ST,Street and Sidewalk Cleaning,Street_Cleaning,Garbage,Voice In,...,4643.0,NaN,6977.0,40.00,39.60,92455.0,1047.0,18297.0,11623.0,2009-06-01
19961,441383,2009-06-01 07:07:34,2009-06-02 08:10:45,Closed,DPW Ops Queue,Intersection of GOLDEN GATE AVE and GOUGH ST,Street and Sidewalk Cleaning,Sidewalk_Cleaning,Furniture,Voice In,...,6382.0,NaN,7892.0,44.70,42.60,64841.0,1488.0,36530.0,14275.0,2009-06-01
19966,441393,2009-06-01 07:23:06,2009-06-03 06:11:01,Closed,DPW Ops Queue,"519 STEVENSON ST, SAN FRANCISCO, CA, 94103",Street and Sidewalk Cleaning,Sidewalk_Cleaning,Garbage,Voice In,...,4643.0,NaN,6977.0,40.00,39.60,92455.0,1047.0,18297.0,11623.0,2009-06-01
20009,441488,2009-06-01 09:23:58,2009-06-11 07:10:53,Closed,DPW Ops Queue,"160 PINE ST, SAN FRANCISCO, CA, 94111",Street and Sidewalk Cleaning,Sidewalk_Cleaning,Sidewalk_Cleaning,Voice In,...,5602.0,NaN,8189.0,46.35,43.65,63703.0,1050.0,13832.5,13794.0,2009-06-01


In [33]:
pride_merged_start = JuneRequests.merge(right = pride, right_on="StartNoTime", left_on="DateOpened")
pride_merged_end = JuneRequests.merge(right = pride, right_on="EndNoTime", left_on="DateOpened")
cols = ["DateOpened", "ReqCount", "attendance_num", "StartNoTime", "EndNoTime"]
pride_merged_start = pride_merged_start[cols]
pride_merged_end = pride_merged_end[cols]
pride_merged = pd.concat([pride_merged_start, pride_merged_end])
pride_merged["Year"] = [t.year for t in pride_merged.StartNoTime]
req_year = pride_merged.groupby(by="Year").sum()
req_year.reset_index(inplace=True)
pride_merged_final = pride_merged_start
pride_merged_final["Year"] = [t.year for t in pride_merged_final.StartNoTime]
pride_merged_final = pride_merged_final.merge(req_year, on = "Year")
pride_merged_final = pride_merged_final[["DateOpened", "ReqCount_y", "attendance_num_x", "Year","StartNoTime", "EndNoTime"]]
pride_merged_final

,DateOpened,ReqCount_y,attendance_num_x,Year,StartNoTime,EndNoTime
0,2009-06-27,39,1200000.0,2009,2009-06-27,2009-06-28
1,2010-06-26,61,1200000.0,2010,2010-06-26,2010-06-27
2,2011-06-25,63,1000000.0,2011,2011-06-25,2011-06-26
3,2012-06-23,56,NaN,2012,2012-06-23,2012-06-24
4,2013-06-29,37,1500000.0,2013,2013-06-29,2013-06-30
5,2014-06-28,62,1700000.0,2014,2014-06-28,2014-06-29
6,2015-06-27,68,1800000.0,2015,2015-06-27,2015-06-28
7,2016-06-25,108,NaN,2016,2016-06-25,2016-06-26


In [43]:
pride_merged_final.to_csv("pride.csv")